In [2]:
#LIBRARIES
import pandas as pd
import re
import numpy as np

## nomalice parameters
from unidecode import unidecode

import spacy  # For preprocessing
from scipy import spatial
from time import time  # To time our operations
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

from tqdm import tqdm

from deep_translator import GoogleTranslator
nlp = spacy.load("en_core_web_sm")

In [3]:
#FUNCIÓN PARA TRADUCIR

def translateString(to_translate, languageFinal):
    translated = GoogleTranslator(source='auto', target=languageFinal).translate(to_translate)
    return translated

#FUNCIÓN PARA LIMPIAR

def cleaning(text):
    text = text.lower()
    text = re.sub("[^A-Za-z']+", ' ', str(text))
    doc = nlp(text)
    return doc



In [4]:
# LEER DATAFRAME Y EXTRAER JOB HISTORY

df = pd.read_excel("2_final_db.xlsx")
df = df [["id", "job_history"]]
df = df[df.job_history.notna()]

In [5]:
#SEPARANDO LOS STRINGS DE CADA TRABAJO

L = []

for i in df.job_history:
    l = str(i).split("},") #convertir en string el job history del individuo y crear lista para separar cada formación
    k = [] #Lista donde se almacenara cada formación x del individuo i
    for job in l: #para cada trabajo
        job = job.replace("[", "").replace("]", "") + "}" #separelos y conviertalas en diccionarios
        k.append(job) #añadalas a la lista de jobs
    L.append(k) 

In [14]:
#EXTRAER DE MANERA ORDENADA EL JOB HISTORY, LA FECHA DE TERMINACIÓN Y COMIENZO

K = [] 

for individuo in tqdm(L): 

    educacion_ind = [] 

    for ins in individuo: 
        ins = ins.replace('"', "'")

        c = {}

        #EXTRAER JOB-TITLE
        if ( len(re.findall("'job_title': '.*?'", ins)) ) > 0:
            c["job_title"] =  re.findall("'job_title': '.*?'", ins)[0].replace("'job_title': ", "").replace("'", "")
            c["job_title"] = str(c["job_title"])
            # c["job_title"] = cleaning(c["job_title"])

        #EXTRAER INICIO
        if ( len(re.findall("'start': '.*?'", ins)) ) > 0:
            c["start"] = re.findall("'start': '.*?'", ins)[0].replace("'start': ", "").replace("'", "")
            c["start"] = int(c["start"][::-1][0:4][::-1]) #SOLO CONSERVAR EL AÑO 

        #EXTRAER FINALIZACIÓN
        if ( len(re.findall("'end': '.*?'", ins)) ) > 0:
            c["end"] = re.findall("'end': '.*?'", ins)[0].replace("'end': ", "").replace("'", "")
            c["end"] = int(c["end"][::-1][0:4][::-1]) #SOLO CONSERVAR EL AÑO

        educacion_ind.append(c) #añado cada formación a la lista de sus educaciones
    
    K.append(educacion_ind) #añado la educación de cada individuo a una lista general

100%|██████████| 389/389 [00:00<00:00, 9046.72it/s]


In [20]:
#TRADUCIR Y LIMPIAR 

for i in tqdm(K[:]):
    for e in tqdm(i):
        # A = "job_title" in e
        # if len(e["job_title"])>0 and A == True:
        try:
            e["job_title"] = translateString(str(e["job_title"]), "en")
            e["job_title"] = cleaning(e["job_title"])
        except:
            None
        

100%|██████████| 389/389 [38:11<00:00,  5.89s/it]


In [22]:
#CREAR LA COLUMNA EN EL DATAFRAME
df["job_history_clean"] = K

In [27]:
df.to_excel("final_db_jh_clean.xlsx", index = False)